# Refer the notebook on Siamese Networks, Batch Normalisation and Skip Connections before using this notebook

# Why [Dropout](http://jmlr.org/papers/v15/srivastava14a.html)


## Regularize Training.

## Helps in generalization.

## ResNets

In [ ]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://github.com/shagunsodhani/PyDataConf2017/blob/master/assets/dropout.jpeg")

In [2]:
import pandas as pd
import numpy as np
np.random.seed(1337)

In [3]:
path_to_dataset = "/home/shagun/FortKnox/Quora/quora_duplicate_questions.tsv"
path_to_glove_vectors = "/home/shagun/models/GloVe/glove.6B.100d.txt"

In [4]:
max_len = 50
embedding_dim = 100
# Refer the exploratory notebook to see how max_len value is arrived at

In [5]:
# Load the dataset into a pandas dataframe
df = pd.read_csv(path_to_dataset, delimiter="\t")
print("Total number of question pairs = ", str(len(df)))

Total number of question pairs =  404290


In [6]:
# Let us look at a sample of the dataset
df_sample = df.sample(5)

df_sample

,id,qid1,qid2,question1,question2,is_duplicate
289783,289783,410957,410958,What are some important international news tha...,What are some instances of your adult cat acti...,0
175085,175085,107022,122068,What is GST bill and what are its implications?,What is this GST bill all about?,0
16059,16059,30638,30639,Does it hurt horses to walk on paved roads?,Does it hurt when horses walk on paved roads?,1
180763,180763,242308,11353,How Can I lose more weight from my face?,How should I lose weight from my face?,1
113420,113420,53324,12719,What are the safety precautions on handling sh...,What are the safety precautions on handling sh...,1


## We will play with a very small sample of this dataset to save on time. Feel free to train the network on the entire data later.

In [7]:
df = df.sample(10000)

In [8]:
labels = list(df['is_duplicate'].apply(lambda x: int(x)).values)
labels = np.asarray(labels)

In [9]:
# Create a list of all the question pairs
first_question_list = list(df['question1'].apply(lambda x: str(x)).values)
second_question_list = list(df['question2'].apply(lambda x: str(x)).values)
question_list = list(zip(first_question_list, second_question_list))

In [10]:
print(len(question_list))

10000


In [11]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/shagunsodhani/PyDataConf2017/master/assets/keras-tensorflow-logo.jpg")
# Image taken from: https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html

In [21]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from utils.util import *
from keras.layers.wrappers import Bidirectional
from keras.layers import Embedding, Input, GRU, Dense, Activation, Lambda, BatchNormalization, Dropout
from keras.layers.merge import add, concatenate
from keras.optimizers import Adam

### Preprocess the data

In [22]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(first_question_list + second_question_list)

In [23]:
sequence1 = pad_sequences(tokenizer.texts_to_sequences(first_question_list), maxlen=max_len)
sequence2 = pad_sequences(tokenizer.texts_to_sequences(second_question_list), maxlen=max_len)

In [24]:
if(len(sequence1) == len(sequence2)):
    print("Good to go")

Good to go


In [25]:
# Basic ML Preprocessing
indices = np.arange(len(sequence1))
np.random.shuffle(indices)
sequence1 = sequence1[indices]
sequence2 = sequence2[indices]
labels = labels[indices]
nb_validation_samples = int(0.3 * len(sequence1))

sequence1_train = sequence1[:-nb_validation_samples]
sequence2_train = sequence2[:-nb_validation_samples]
labels_train = labels[:-nb_validation_samples]
sequence1_val = sequence1[-nb_validation_samples:]
sequence2_val = sequence2[-nb_validation_samples:]
labels_val = labels[-nb_validation_samples:]

In [26]:
print("Number of training examples: " + str(len(sequence1_train)))

Number of training examples: 7000


In [27]:
print("Number of validation examples: " + str(len(sequence1_val)))

Number of validation examples: 3000


### Load embeddings

In [28]:
# Load embeddings
embeddings_index = get_glove_embeddings(path_to_glove_vectors)

### Preparing the embedding matrix which our model would use

In [29]:
# Preparing the embedding matrix which our model would use
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

### Encode the question into a vector space using a Bidirectional GRU (or LSTM or whatever RNN you believe in)

In [30]:
def question_encoder():
    return Bidirectional(GRU(units=200), merge_mode='concat', name="bidir_gru")

### The next part is the core of this network and we would walk through it slowly

In [31]:
def create_question_network():
    
#     Create an input layer
    sequence_input = Input(shape=(max_len,), dtype='int32', name="input_layer")
    
#     Create an embedding layer
    embedding_layer = Embedding(len(word_index) + 1,
                            embedding_dim,
                            weights=[embedding_matrix],
                            input_length=max_len,
                            mask_zero = True,
                            trainable=False)
    
#     Use the embedding layer we just created
    embedded_sequences = embedding_layer(sequence_input)
    
#     Embeddings are for words, sentences uses encoders
    encoded_question = question_encoder()(embedded_sequences)
    
    dropout_prob = 0.4
#     Lets fully connect them
    dense1 = Dense(64)(encoded_question)
#     Dropout
    dense1 = Dropout(dropout_prob)(dense1)
#     BatchNormalization
    bn1 = BatchNormalization()(dense1)
    relu1 = Activation('relu')(bn1)
    
#    Make it deep
    dense2 = Dense(64)(relu1)
#     Dropout
    dense2 = Dropout(dropout_prob)(dense2)
#     BatchNormalization
    bn2 = BatchNormalization()(dense2)
#     Skip Connection
    res2 = add([relu1, bn2])
    relu2 = Activation('relu')(res2)    

#     And Deeper
    dense3 = Dense(64)(relu2)
#     Dropout
    dense3 = Dropout(dropout_prob)(dense3)
#     BatchNormalization
    bn3 = BatchNormalization()(dense3)
#     Skip Connection
    res3 = add([relu2, bn3])
    relu3 = Activation('relu')(res3)

#     Now we are in rythm
    dense4 = Dense(64)(relu3)
#     Dropout
    dense4 = Dropout(dropout_prob)(dense4)
#     BatchNormalization
    bn4 = BatchNormalization()(dense4)
#     Skip Connection
    res4 = add([relu3, bn4])
    relu4 = Activation('relu')(bn4)

    features = concatenate([relu4, relu3, relu2, relu1])
#     BatchNormalization
    output = BatchNormalization()(features)
    
    model = Model(inputs=sequence_input, outputs=output)

    return model

### Now we will make the siamese twin

In [32]:
def create_network():
    # network definition
    question_network = create_question_network()
    
#     input to the first head of the network
    input1 = Input(shape=(max_len,))
    
#     input to the second head of the network
    input2 = Input(shape=(max_len,))
    
#     processing the first input
    processed1 = question_network(input1)
    
#     processing the second input
    processed2 = question_network(input2)
    
#     Computing the distance between the transformed inputs.
    distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed1, processed2])

    
    model = Model(inputs=[input1, input2], outputs=distance)
    return model

In [33]:
model = create_network()

optimizer = Adam(lr=0.001, clipnorm=5)
model.compile(loss=contrastive_loss, optimizer=optimizer)

for i in range(10):
    model.fit([sequence1_train, sequence2_train], labels_train,
       validation_data=([sequence1_val, sequence2_val], labels_val),
       batch_size=128, epochs=1)
    
    model_labels_train = model.predict([sequence1_train, sequence2_train], batch_size=128)
    print("Training accuracy: "+str(compute_accuracy(model_labels_train, labels_train)))
    
    model_labels_val = model.predict([sequence1_val, sequence2_val], batch_size=128)
    print("Validation accuracy: "+str(compute_accuracy(model_labels_val, labels_val)))

Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 124s - loss: 155.8288 - val_loss: 11.4491
Training accuracy: 0.622571428571
Validation accuracy: 0.627666666667
Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 161s - loss: 113.7380 - val_loss: 184.5496
Training accuracy: 0.624714285714
Validation accuracy: 0.628666666667
Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 132s - loss: 85.5662 - val_loss: 17.7574
Training accuracy: 0.621714285714
Validation accuracy: 0.623333333333
Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 125s - loss: 67.9190 - val_loss: 23.4337
Training accuracy: 0.620428571429
Validation accuracy: 0.624
Train on 7000 samples, validate on 3000 samples
Epoch 1/1
7000/7000 [==============================] - 153s - loss: 54.8989 - val_loss: 19.6960


KeyboardInterrupt: 

In [ ]:
model_labels_val.ravel() < 0.5

In [ ]:
np.mean(np.equal(model_labels_val.ravel() < 0.5, labels_val))

In [ ]:
compute_accuracy()